# Content-based Recommendation
(by Tevfik Aytekin)

Content-based recommender algorithms use the content of the items for making a recommendation. For example, for movie recommendation movie contents such as plot summary, director, casting, jenres, release date, etc. and user content such as previously watched movies, gender, age, etc. can be used to find out which movies can be recommended to the users.
(by Tevfik Aytekin)

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from scipy.sparse import csr_matrix
from collections import Counter
from sklearn.metrics import pairwise_distances
from operator import itemgetter
import copy
import re
import nltk
import heapq
import sys, os
import pickle
import itertools
import operator
from tqdm.notebook import tqdm
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import FunctionTransformer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

### Movielens ml-latest-small dataset

In [4]:
with open('../datasets/ml-latest-small/README.txt', 'r') as f:
    print(f.read())

Summary

This dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. It contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018.

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in the files `links.csv`, `movies.csv`, `ratings.csv` and `tags.csv`. More details about the contents and use of all these files follows.

This is a *development* dataset. As such, it may change over time and is not an appropriate dataset for shared research results. See available *benchmark* datasets if that is your intent.

This and other GroupLens data sets are publicly available for down

In [5]:
ratings = pd.read_csv("../datasets/ml-latest-small/ratings.csv", sep=",")
print(ratings.shape)
ratings.head()

(100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
links = pd.read_csv("../datasets/ml-latest-small/links.csv", sep=",")
print(links.shape)
links.head()

(9742, 3)


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [7]:
movies = pd.read_csv("../datasets/ml-latest-small/movies.csv", sep=",")
print(movies.shape)
movies.head()

(9742, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
tags = pd.read_csv("../datasets/ml-latest-small/tags.csv", sep=",")
print(tags.shape)
tags.head()

(3683, 4)


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [9]:
imdb_details = pd.read_json("../datasets/ml-latest-small/IMDB_movie_details.json", lines=True)
print(imdb_details.shape)
imdb_details.head()

(1572, 7)


,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in..."
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...
2,tt0243655,"The setting is Camp Firewood, the year 1981. I...",1h 37min,"[Comedy, Romance]",6.7,2002-04-11,
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...


## Create User Item Rating Map
It might take some time but will be useful later.

In [10]:
rating_map = {}
for i in range(len(ratings)):
    key = str(ratings.iloc[i,0]) + '_' +str(ratings.iloc[i,1])
    rating_map[key]=ratings.iloc[i,2]

In [11]:
rating_map["1_47"]

5.0

In [12]:
iterator = iter(rating_map.items())
for i in range(5):
    print(next(iterator))

('1_1', 4.0)
('1_3', 4.0)
('1_6', 4.0)
('1_47', 5.0)
('1_50', 5.0)


## Create movie tags map
This map will also be useful later

In [13]:
movie_tags = {}
for i in range(len(tags)):
    key = tags.iloc[i,1]
    if key in movie_tags:
        movie_tags[key].append(tags.iloc[i,2])
    else:
        movie_tags[key] = [tags.iloc[i,2]]

In [14]:
movie_tags

{60756: ['funny',
  'Highly quotable',
  'will ferrell',
  'comedy',
  'funny',
  'will ferrell',
  'funny',
  'will ferrell'],
 89774: ['Boxing story', 'MMA', 'Tom Hardy'],
 106782: ['drugs',
  'Leonardo DiCaprio',
  'Martin Scorsese',
  'Stock Market',
  'Wall Street'],
 48516: ['way too long',
  'Leonardo DiCaprio',
  'suspense',
  'twist ending',
  'undercover cop',
  'atmospheric',
  'Jack Nicholson',
  'Leonardo DiCaprio',
  'Martin Scorsese',
  'suspense'],
 431: ['Al Pacino', 'gangster', 'mafia'],
 1221: ['Al Pacino', 'Mafia', 'Mafia'],
 5995: ['holocaust', 'true story', 'Holocaust'],
 44665: ['twist ending'],
 52604: ['Anthony Hopkins', 'courtroom drama', 'twist ending'],
 88094: ['britpop', 'indie record label', 'music'],
 144210: ['dumpster diving', 'Sustainability'],
 1569: ['romantic comedy', 'wedding', 'weddings'],
 118985: ['painter'],
 119141: ['bloody',
  'bromance',
  'comedy',
  'funny',
  'James Franco',
  'Seth Rogen'],
 109487: ['black hole',
  'sci-fi',
  'time-t

In [15]:
iterator = iter(movie_tags.items())
for i in range(5):
    print(next(iterator))

(60756, ['funny', 'Highly quotable', 'will ferrell', 'comedy', 'funny', 'will ferrell', 'funny', 'will ferrell'])
(89774, ['Boxing story', 'MMA', 'Tom Hardy'])
(106782, ['drugs', 'Leonardo DiCaprio', 'Martin Scorsese', 'Stock Market', 'Wall Street'])
(48516, ['way too long', 'Leonardo DiCaprio', 'suspense', 'twist ending', 'undercover cop', 'atmospheric', 'Jack Nicholson', 'Leonardo DiCaprio', 'Martin Scorsese', 'suspense'])
(431, ['Al Pacino', 'gangster', 'mafia'])


In [16]:
movie_tags[3]

['moldy', 'old']

## Create movie plot summary map
This map will also be useful later

In [17]:
movie_ps = {}
for i in range(len(imdb_details)):
    key = imdb_details.iloc[i,0]
    key = str.replace(key, "tt","")
    key = str.replace(key, "/","")
    key = int(key)
    movie_ps[key] = imdb_details.iloc[i,1]

# joined_summaries= movies.merge(movie_ps, left_on="movieId", right_on="movie_id")

In [18]:
movie_ps

{105112: "Former CIA analyst, Jack Ryan is in England with his family on vacation when he suddenly witnesses an explosion outside Buckingham Palace. It is revealed that some people are trying to abduct a member of the Royal Family but Jack intervenes, killing one of them and capturing the other, and stops the plan in its tracks. Afterwards, he learns that they're Irish revolutionaries and the two men are brothers. During his court hearing the one that's still alive vows to get back at Jack but is sentenced and that seems to be the end of it. However, whilst the man is being transported, he is broken out. Jack learns of this but doesn't think there's anything to worry about. But, when he is at the Naval Academy someone tries to kill him. He learns that they are also going after his family and so he rushes to find them, safe but having also been the victims of a failed assassination. That's when Jack decides to rejoin the CIA, and they try to find the man before he makes another attempt.

In [19]:
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
              
stemmer = SnowballStemmer("english", ignore_stopwords=False)
def normalized(X): 
  normalized = []
  for x in X:
    words = nltk.word_tokenize(x)
    normalized.append(' '.join([stemmer.stem(word) for word in words if re.match('[a-zA-Z]+', word)]))
  return normalized

pipe = Pipeline([
  ('normalize', FunctionTransformer(normalized, validate=False)),
  ('counter_vectorizer', CountVectorizer(
    max_df=0.8, max_features=200000,
    min_df=0.2, stop_words='english',
    ngram_range=(1,3)
  )),
  ('tfidf_transform', TfidfTransformer())
])

tfidf_matrix = pipe.fit_transform([x for x in movie_ps.values()])
similarity_distance = 1 - cosine_similarity(tfidf_matrix)

In [20]:
similarity_distance

array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 0.        , ..., 1.        , 0.53928726,
        1.        ],
       ...,
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 0.53928726, ..., 1.        , 0.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ]])

In [93]:
prediction = (1/8*4+1/5*4+2/5*4+1/5*5+1/6*5)/(1/8+1/5+2/5+1/5+1/6)
prediction

4.335877862595419

### Jaccard Similarity

Given two sets $A$ and $B$,

$Jaccard(A, B) = \frac{|A \cap B|}{|A \cup B|}$

For example if $A = \{a, b, c, d\}$ and $B = \{b, d, e ,f, g\}$ then

$Jaccard(A, B) = \frac{2}{7}$

In [21]:
# finds the tags similary of items i and j using Jaccard similarity
def tags_sim(i,j):
    if i not in movie_tags or j not in movie_tags:
        return 0

    tags_i = movie_tags[i]
    tags_j = movie_tags[j]
    # print(tags_i)
    # print(tags_j)
    intersection_size = len(set(tags_i).intersection(tags_j))
    union_size = len(set(tags_i).union(tags_j))
    return intersection_size / union_size
    

In [22]:
def ps_sim(i,j):
    if i not in movie_ps or j not in movie_ps:
        return 0

    print(i)
    
    ps_i = movie_ps[i]
    ps_j = movie_ps[j]

    # tokenization 
    X_list = word_tokenize(ps_i)  
    Y_list = word_tokenize(ps_j) 

    # sw contains the list of stopwords 
    sw = stopwords.words('english')  
    l1 =[];l2 =[] 

    # remove stop words from the string 
    X_set = {w for w in X_list if not w in sw}  
    Y_set = {w for w in Y_list if not w in sw} 

    # form a set containing keywords of both strings  
    r_vector = X_set.union(Y_set)  
    for w in r_vector: 
        if w in X_set: l1.append(1) # create a vector 
        else: l1.append(0) 
        if w in Y_set: l2.append(1) 
        else: l2.append(0) 
    c = 0
    for i in range(len(r_vector)): 
        c+= l1[i]*l2[i] 
    cosine = c / float((sum(l1)*sum(l2))**0.5) 
    return cosine

In [23]:
def content_based_similar_movies(i):
    sim_movies = []
    for j in movie_tags:
        if i == j:
            continue
        sim = tags_sim(i, j)
        if sim > 0:
            sim_movies.append({"movieId":j, "sim": sim})
    sim_movies.sort(key=lambda sim_movies:sim_movies["sim"], reverse=True)
    return sim_movies

In [24]:
content_based_similar_movies(5)

[{'movieId': 7, 'sim': 0.5},
 {'movieId': 2719, 'sim': 0.5},
 {'movieId': 4808, 'sim': 0.5},
 {'movieId': 6788, 'sim': 0.5},
 {'movieId': 34359, 'sim': 0.5},
 {'movieId': 34528, 'sim': 0.5},
 {'movieId': 1367, 'sim': 0.3333333333333333},
 {'movieId': 2424, 'sim': 0.3333333333333333},
 {'movieId': 6944, 'sim': 0.3333333333333333},
 {'movieId': 8366, 'sim': 0.3333333333333333},
 {'movieId': 5064, 'sim': 0.25},
 {'movieId': 1343, 'sim': 0.16666666666666666},
 {'movieId': 56367, 'sim': 0.16666666666666666},
 {'movieId': 32, 'sim': 0.1111111111111111}]

In [25]:
def content_based_similar_movies_all():
    sim_movies = {}
    for i in movie_tags:
        i_sim = content_based_similar_movies(i)
        sim_movies[i]= i_sim
                    
    return sim_movies

In [26]:
content_based_similar_movies_all()

{60756: [{'movieId': 107348, 'sim': 0.3333333333333333},
  {'movieId': 69122, 'sim': 0.2857142857142857},
  {'movieId': 119141, 'sim': 0.25},
  {'movieId': 126548, 'sim': 0.2},
  {'movieId': 3948, 'sim': 0.2},
  {'movieId': 6188, 'sim': 0.2},
  {'movieId': 167746, 'sim': 0.2},
  {'movieId': 88405, 'sim': 0.18181818181818182},
  {'movieId': 183611, 'sim': 0.16666666666666666},
  {'movieId': 101142, 'sim': 0.16666666666666666},
  {'movieId': 134170, 'sim': 0.16666666666666666},
  {'movieId': 148626, 'sim': 0.16666666666666666},
  {'movieId': 2953, 'sim': 0.14285714285714285},
  {'movieId': 61024, 'sim': 0.14285714285714285},
  {'movieId': 179401, 'sim': 0.14285714285714285},
  {'movieId': 193565, 'sim': 0.14285714285714285},
  {'movieId': 106766, 'sim': 0.14285714285714285},
  {'movieId': 112852, 'sim': 0.14285714285714285},
  {'movieId': 8641, 'sim': 0.1111111111111111},
  {'movieId': 68848, 'sim': 0.1111111111111111},
  {'movieId': 71535, 'sim': 0.1111111111111111},
  {'movieId': 4816,

In [27]:
def content_based_rating_prediction(u, i):
    r = 0
    sum_sim = 0
    # find the movies rated by u
    movies = ratings[ratings["userId"]==u].movieId
    for j in movies:
        sim = ps_sim(i, j)
        key = str(u)+"_"+str(j)
        r += sim*rating_map[key]
        sum_sim += sim
    if sum_sim == 0:
        return 0
    else:
        return r / sum_sim       

In [28]:
content_based_rating_prediction(5,1)

0

## Evaluation of Rating Prediction

How can we measure the performance of a recommender algorithm? This is similar to the evaluation used in machine learning.

- Make a train/test split
- Build the model on the training set
- Make predictions for the ratings in the test set
- Find the mean absolute error (MAE)

For more metrics other then MAE look at the "Metrics for Regression" section of [this notebook](http://localhost:8888/notebooks/PycharmProjects/data_science/evaluation.ipynb)


In [30]:
nltk.download('stopwords')

X_train, X_test = train_test_split(ratings, test_size=1000)
train_size = X_train.shape[0]
test_size = X_test.shape[0]
print("Test size:", test_size)
error = 0
for k in range(test_size): 
    u = X_test.iloc[k,0]
    i = X_test.iloc[k,1]
    r = X_test.iloc[k,2]
    error += np.abs(r - content_based_rating_prediction(u,i))
print(error/test_size)

[nltk_data] Downloading package stopwords to /home/tx/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Test size: 1000
106918
106918
86190
86190
86190
106489
106489
106489
3.4533976637143566


## Top-N recommendation Algorithm - Predict and Sort
The task in top-$N$ recommendation is to recommend $N$ items to a user. 


Recommend $N$ movies to user $u$
- Predict the ratings of all items which are not watched by $u$
- Sort the predicted ratings
- Recommend the movies with the highest predicted ratings

In [31]:
def top_N_pred_sort(N, u):
    preds = pd.Series([], dtype='float')
    # find the movies not rated by u
    movies_not_rated = ratings.query("userId != @u").movieId.unique()
    for m in movies_not_rated:
        preds[m] = content_based_rating_prediction(u, m)
    return preds.sort_values(ascending=False)[:N]    

In [32]:
top_N_pred_sort(10, 1)

163981    0
318       0
333       0
1704      0
3578      0
6874      0
8798      0
46970     0
48516     0
58559     0
dtype: int64

## Efficiency Issues

There are important inefficiencies in this algorithm:

- The algorithm predicts the rating of all items which are not rated by the user. In the case of millions of items this algorithm is practically infeasible. Numerous techniques have been developed to remedy this problem. Can you suggest a solution? 
- In rating prediction, similarity between target item and items rated by the user are calculated. To make a recommendation to another user similarity calculations will be done again. For making recommendations to users in general many similarity calculations will be repeated. A general solution to this problem is to precalculate the similarities between items. Moreover, you don't need to store all similarities, only storing $k$ most similar items to every item will be enough. Size of $k$ can be determined according to the needs.


## Top-N recommendation Algorithm - kNN Map
The task in top-$N$ recommendation is to recommend $N$ items to a user. 

- Build a knn-map (a map which stores the $k$ nearest neighbors of each item)

Recommend $N$ movies to user $u$
- Get the neigbors of movies which are watched by $u$ and put them into a list $C$.
- Choose $N$ movies from $C$. There can be different methods here. Most repeated movies in C can be chosen, movies with the highest total similarity (or maximum similarity) can be chosen.
- Recommend the $N$ movies that are chosen.

## Building a knn map
This table will hold the most similar $k$ items for each item. In order to build this table we need to calculate all pairwise similarities which takes $O(n^2)$ time. There is no escape from this $O(n^2)$ time unless you use an approximation algorithm such as LSH (Locality Sensitive Hashing) for nearest neighbor search.

We will use a heap based priority queue for storing the nearest neighbors. You can look at this [animation](https://www.cs.usfca.edu/~galles/visualization/Heap.html).

In [33]:
pq =[(10,"a"),(8, "b"), (5, "c")]
heapq.heapify(pq)
heapq.nsmallest(2,pq)

[(5, 'c'), (8, 'b')]

In [ ]:
movies[:10]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [34]:
def build_knn_map(movies, K=10):
    knn_map = {}
    movie_ids = movies['movieId'].unique()
    for i in tqdm(range(len(movie_ids))):
        pq = []
        knn_map[movie_ids[i]] = pq
        for j in range(len(movie_ids)):
            if (i == j):
                continue
            sim = tags_sim(movie_ids[i],movie_ids[j])
            if (len(pq) >= K):
                smallest = pq[0]
                if (sim > smallest[0]):
                    heapq.heappop(pq)
                    heapq.heappush(pq, (sim, movie_ids[j]))
            else:
                heapq.heappush(pq, (sim, movie_ids[j]))
    return knn_map

In [35]:
knn_map = build_knn_map(movies,K=30)

  0%|          | 0/9742 [00:00<?, ?it/s]

In [36]:
knn_map[1]

[(0, 6),
 (0, 9),
 (0.0, 7),
 (0, 10),
 (0.0, 11),
 (0, 13),
 (0, 8),
 (0.0, 17),
 (0, 19),
 (0.0, 21),
 (0, 12),
 (0.0, 25),
 (0.0, 14),
 (0, 15),
 (0.0, 16),
 (0.0, 31),
 (0, 18),
 (0.125, 108932),
 (0, 20),
 (0.125, 89745),
 (0.0, 22),
 (0, 23),
 (0, 24),
 (0.005747126436781609, 296),
 (0.0, 26),
 (0, 27),
 (0.0, 28),
 (0.0, 29),
 (0, 30),
 (0.5, 122918)]

## Top-N recommendation using knn map

In [37]:
def add_sims_and_sort(l):
    li = []
    it = itertools.groupby(l, operator.itemgetter(1))
    for key, subiter in it:
        li.append((key, sum(item[0] for item in subiter)))
    li = sorted(li, key=itemgetter(0), reverse=True)
    return li
    

In [38]:
def top_N_knn_map(ratings, N, u):
    C = []
    # find the movies rated by u
    movies_rated = ratings.query("userId == @u").movieId
    for m in movies_rated:
        C = C + knn_map[m]
    return add_sims_and_sort(C)[:N]    

In [39]:
top_N_knn_map(ratings, 10, 1)

[(184471, 0.125),
 (180031, 0.1),
 (179401, 0.1),
 (176371, 0.05660377358490566),
 (174055, 0.1),
 (174055, 0.1111111111111111),
 (168252, 0.07692307692307693),
 (168248, 0.04),
 (168248, 0.08333333333333333),
 (167746, 0.1)]

## Evaluation of top-N recommendation 

Evaluation of rating prediction is rather easy: find the mean absolute error between rating predictions and true ratings. How can we evaluate the accuracy of a top-N recommendation? There are several techniques which we will look at in more detail later. Below is one common way to evaluate top-N recommendation:

- Randomly sub-sample some portion of positive preferences in order to create a test set $T$. Positive preferences might be 5-star ratings, movies watched more than a certain threshold, or items purchased.
- Put the rest of the preferences into the training set and build model.

- For each preference $(u,i)$ in the test set:
    - Make a top-N recommendation tu user $u$.
    - If the test item i occurs among the top-N items, then we have a hit, otherwise we have a miss. 

Hit ratio is then defined as: 

$$
Hit Ratio: \frac{\#hits}{|T|}
$$



In [40]:
N = 1000
X_train, X_test = train_test_split(ratings, test_size=1000)
X_test = X_test.query("rating > 4")
train_size = X_train.shape[0]
test_size = X_test.shape[0]
print("Test size:", test_size)
hit_count = 0
for k in range(test_size): 
    u = X_test.iloc[k,0]
    i = X_test.iloc[k,1]
    r = X_test.iloc[k,2]
    top_N = top_N_knn_map(X_train, N, u)
    hit_list = [item for item in top_N if item[0] == i]
    if len(hit_list) > 0:
        hit_count +=1
print("Hit Ratio", hit_count/test_size)

Test size: 223
Hit Ratio 0.15246636771300448
